### Converting a SavedModel
The following example shows how to convert a SavedModel into a TensorFlow Lite FlatBuffer.

In [21]:
import tensorflow as tf

# Construct a basic model.
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2 * x)

# Save the model.
export_dir = '/tmp/test_saved_model'
input_data = tf.constant(1., shape=[1, 1])
to_save = root.f.get_concrete_function(input_data)
tf.saved_model.save(root, export_dir, to_save)

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/test_saved_model/assets


This API does not have the option of specifying the input shape of any input arrays. If your model requires specifying the input shape, use the `from_concrete_functions` classmethod instead. The code looks similar to the following:

### Converting a Keras model
The following example shows hot to convert a `tf.keras` model into TensorFlow Lite `FlatBuffer`

In [4]:
# Create a simple Keras model.
x = [-1, 0, 1, 2, 3, 4]
y = [-3, -1, 1, 3, 5, 7]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=[1])])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(x, y, epochs=50)

Train on 6 samples
Epoch 1/50
6/6 [==============================] - 0s 29ms/sample - loss: 36.1476
Epoch 2/50
6/6 [==============================] - 0s 1ms/sample - loss: 28.7784
Epoch 3/50
6/6 [==============================] - 0s 2ms/sample - loss: 22.9736
Epoch 4/50
6/6 [==============================] - 0s 3ms/sample - loss: 18.3998
Epoch 5/50
6/6 [==============================] - 0s 2ms/sample - loss: 14.7948
Epoch 6/50
6/6 [==============================] - 0s 3ms/sample - loss: 11.9519
Epoch 7/50
6/6 [==============================] - 0s 2ms/sample - loss: 9.7089
Epoch 8/50
6/6 [==============================] - 0s 2ms/sample - loss: 7.9379
Epoch 9/50
6/6 [==============================] - 0s 2ms/sample - loss: 6.5384
Epoch 10/50
6/6 [==============================] - 0s 2ms/sample - loss: 5.4314
Epoch 11/50
6/6 [==============================] - 0s 2ms/sample - loss: 4.5545
Epoch 12/50
6/6 [==============================] - 0s 2ms/sample - loss: 3.8588
Epoch 13/50
6/6 [======

In [7]:
# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

In [8]:
open('converted_model.tflite', 'wb').write(tflite_model)

732

### Converting a concrete function
The following example shows how to convert a TensorFlow `concrete function` into a TensorFlow Lite `FlatBuffer`.

In [9]:
# Construct a basic model.
root = tf.train.Checkpoint()
root.v1 = tf.Variable(3.)
root.v2 = tf.Variable(2.)
root.f = tf.function(lambda x: root.v1 * root.v2)

# Create the concrete function.
input_data = tf.constant(1., shape=[1, 1])
concrete_func = root.f.get_concrete_function(input_data)

# Convert the model.
# `from_concrete_function` takes a list of concrete functions, however,
# currently only supports converting one function at a time. Converting multiple
# functions is under development.
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
tflite_model = converter.convert()

### End-to-end MobileNet conversion
The following example shows hot to convert and run inference on pre-trained `tf.keras` MobileNet model to TensorFlow Lite.
It compares the results of the TensorFlow and TensorFlow Lite model on random data. In order to load the model from file, use `model_path` instead of `model_content`

In [10]:
# Load the MobileNet tf.keras.model.
model = tf.keras.applications.MobileNetV2(
    weights='imagenet', input_shape=(224, 224, 3))

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors() # ???

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print(input_details)

14540800/14536120 [==============================] - 2s 0us/step
[{'name': 'input_1', 'index': 1, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}]


In [14]:
print(input_details[0])

{'name': 'input_1', 'index': 1, 'shape': array([  1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}


In [15]:
import numpy as np


# Test the TensorFlow Lite model on random input data.
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)
interpreter.set_tensor(input_details[0]['index'], input_data)

In [16]:
interpreter.invoke()

In [17]:
# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [19]:
# Test the TensorFlow model on random input data.
tf_results = model(tf.constant(input_data))

In [20]:
# Compare the result.
for tf_result, tflite_result in zip(tf_results, tflite_results):
    np.testing.assert_almost_equal(tf_result, tflite_result, decimal=5)